In [1]:
# Set env var OPENAI_API_KEY or load from a .env file:
import dotenv

dotenv.load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.2)

In [3]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain.memory import ChatMessageHistory


In [4]:
from langchain_community.document_loaders import WebBaseLoader
#pull data from webpage
loader = WebBaseLoader("https://python.langchain.com/docs/use_cases/chatbots/quickstart")
data = loader.load()

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [6]:
#store chunks in vector database
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())
# ok

In [7]:
#create retriever
# k is the number of chunks to retrieve
retriever = vectorstore.as_retriever(k=4)
docs = retriever.invoke("how can langsmith help with testing?")

docs

[Document(page_content="statistics, and troubleshoot specific issues as they encounter them during testing. This real-time monitoring and troubleshooting capability contribute to the overall effectiveness and reliability of the testing process.\\n\\nIn essence, LangSmith's features are designed to improve the quality and reliability of testing by expanding evaluation sets, fine-tuning models, and providing comprehensive monitoring capabilities. These aspects collectively contribute to a more robust and thorough testing", metadata={'description': 'Overview', 'language': 'en', 'source': 'https://python.langchain.com/docs/use_cases/chatbots/quickstart', 'title': 'Quickstart | 🦜️🔗 Langchain'}),
 Document(page_content='and informative documents:retriever.invoke("how can langsmith help with testing?")[Document(page_content=\'You can also quickly edit examples and add them to datasets to expand the surface area of your evaluation sets or to fine-tune a model for improved quality or reduced co

In [8]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.memory import ChatMessageHistory
from typing import Dict
from langchain_core.runnables import RunnablePassthrough

chat = ChatOpenAI(model="gpt-3.5-turbo-1106")

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

In [9]:

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("how can langsmith help with testing?")

document_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
        "context": docs,
    }
)

'LangSmith can help with testing in several ways:\n\n1. Expansion of Evaluation Sets: LangSmith allows you to quickly edit examples and add them to datasets, expanding the surface area of your evaluation sets. This is essential for a more thorough assessment of model performance and application behavior.\n\n2. Fine-Tuning Models: LangSmith supports the fine-tuning of models to enhance their quality and reduce operational costs. This capability is valuable during testing as it enables the optimization of model performance based on specific testing requirements and objectives.\n\n3. Monitoring: LangSmith provides monitoring functionality, allowing users to log traces, visualize latency, and track token usage for real-time monitoring and troubleshooting during testing. This contributes to the overall effectiveness and reliability of the testing process.'

In [10]:
def parse_retriever_input(params: Dict):
    return params["messages"][-1].content


retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | retriever,
).assign(
    answer=document_chain,
)

In [11]:
response = retrieval_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
    }
)

response

{'messages': [HumanMessage(content='how can langsmith help with testing?')],
 'context': [Document(page_content="statistics, and troubleshoot specific issues as they encounter them during testing. This real-time monitoring and troubleshooting capability contribute to the overall effectiveness and reliability of the testing process.\\n\\nIn essence, LangSmith's features are designed to improve the quality and reliability of testing by expanding evaluation sets, fine-tuning models, and providing comprehensive monitoring capabilities. These aspects collectively contribute to a more robust and thorough testing", metadata={'description': 'Overview', 'language': 'en', 'source': 'https://python.langchain.com/docs/use_cases/chatbots/quickstart', 'title': 'Quickstart | 🦜️🔗 Langchain'}),
  Document(page_content='and informative documents:retriever.invoke("how can langsmith help with testing?")[Document(page_content=\'You can also quickly edit examples and add them to datasets to expand the surfa

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

# We need a prompt that we can pass into an LLM to generate a transformed search query

chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.2)

query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.",
        ),
    ]
)

query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("messages", [])) == 1,
        # If only one message, then we just pass that message's content to retriever
        (lambda x: x["messages"][-1].content) | retriever,
    ),
    # If messages, then we pass inputs to LLM chain to transform the query, then pass to retriever
    query_transform_prompt | chat | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")

In [13]:
document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

conversational_retrieval_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain,
).assign(
    answer=document_chain,
)

demo_ephemeral_chat_history = ChatMessageHistory()

In [14]:
demo_ephemeral_chat_history.add_user_message("how can langsmith help with testing?")

response = conversational_retrieval_chain.invoke(
    {"messages": demo_ephemeral_chat_history.messages},
)

demo_ephemeral_chat_history.add_ai_message(response["answer"])

response

{'messages': [HumanMessage(content='how can langsmith help with testing?'),
  AIMessage(content="LangSmith can help with testing in several ways:\n\n1. Expansion of Evaluation Sets: LangSmith enables the expansion of evaluation sets, allowing for a more thorough assessment of model performance and application behavior. This broadens the range of scenarios and inputs that can be used to assess performance, contributing to the overall effectiveness and reliability of the testing process.\n\n2. Fine-Tuning Models: LangSmith supports the fine-tuning of models to enhance their quality and reduce operational costs. This capability is valuable during testing as it enables the optimization of model performance based on specific testing requirements and objectives.\n\n3. Monitoring: LangSmith provides monitoring functionality, allowing users to log traces, visualize latency, and monitor token usage. This real-time monitoring and troubleshooting capability contribute to the overall effectiveness

In [15]:
demo_ephemeral_chat_history.add_user_message("tell me more about that!")

conversational_retrieval_chain.invoke(
    {"messages": demo_ephemeral_chat_history.messages}
)

{'messages': [HumanMessage(content='how can langsmith help with testing?'),
  AIMessage(content="LangSmith can help with testing in several ways:\n\n1. Expansion of Evaluation Sets: LangSmith enables the expansion of evaluation sets, allowing for a more thorough assessment of model performance and application behavior. This broadens the range of scenarios and inputs that can be used to assess performance, contributing to the overall effectiveness and reliability of the testing process.\n\n2. Fine-Tuning Models: LangSmith supports the fine-tuning of models to enhance their quality and reduce operational costs. This capability is valuable during testing as it enables the optimization of model performance based on specific testing requirements and objectives.\n\n3. Monitoring: LangSmith provides monitoring functionality, allowing users to log traces, visualize latency, and monitor token usage. This real-time monitoring and troubleshooting capability contribute to the overall effectiveness